In [15]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

CONTINENTS = ['Africa', 'Asia', 'Oceania', 'Europe', 'Northern America', 'South America', 'Antarctic Region']
PRIMARY_FORREST_PRODUCTS = ['Roundwood', 'Sawnwood', 'Wood-based panels', 'Paper and paperboard',  'Pulpwood:round and split:all species (production)']
LAND_USE_FOREST_ITEMS = ['Forestry']


In [16]:
# load forestry dataset
forest_all = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/Forestry_E_All_Data_cleared.csv", engine='python')


In [17]:
# filter continets
continent_production = forest_all[forest_all.Element.str.match("Production") & 
                                    forest_all.Area.isin(CONTINENTS) &
                                    forest_all.Item.isin(PRIMARY_FORREST_PRODUCTS)]


In [19]:
# load landuse dataset
land_use_all = pd.read_csv("data/faostat/Inputs_LandUse_E_All_Data_cleared.csv", engine='python')
land_use = land_use_all[land_use_all.Item.isin(LAND_USE_FOREST_ITEMS) &
                        land_use_all.Element.str.match('Area') &
                        land_use_all.Area.isin(CONTINENTS)]

In [20]:
# compute the range of years in the plot
year_min = int(max(max(continent_production.Year.min(), land_use.Year.min()), 1995))
year_max = int(min(min(continent_production.Year.max(), land_use.Year.max()), 2015))


In [23]:
continent_production.Item.unique()

array(['Roundwood', 'Pulpwood:round and split:all species (production)',
       'Sawnwood', 'Wood-based panels', 'Paper and paperboard'],
      dtype=object)

In [27]:
# add traces of each of the production/ forested area

ig = go.Figure()

fig = make_subplots(
    rows=1, 
    cols=2,
    subplot_titles=("Production (m3)", "Forested area (1000 ha)"),
    shared_yaxes=True
)

for year in range(year_min, year_max +1):
    a = continent_production[continent_production.Year == year].groupby(["Area", "Item"]).Value.sum()
    a = a.reset_index()
    a = a[a.Area.str.match("Oceania")].append(a[a.Area.str.match("South America")])\
                                      .append(a[a.Area.str.match("Africa")])\
                                      .append(a[a.Area.str.match("Northern America")])\
                                      .append(a[a.Area.str.match("Europe")])\
                                      .append(a[a.Area.str.match("Asia")])
    fig.add_trace(
        go.Bar(
            y=a[a.Item.str.match("Roundwood")].Area.values,
            x=a[a.Item.str.match("Roundwood")].Value.values,
            name='Roundwood, ' + str(year),
            orientation='h',
            visible=False,
        ),    
        row=1,
        col=1, 
    )
    
    fig.add_trace(
        go.Bar(
            y=a[a.Item.str.match("Wood-based panels")].Area.values,
            x=a[a.Item.str.match("Wood-based panels")].Value.values,
            name='Wood-based panels, ' + str(year),
            orientation='h',
            visible=False,
        ),    
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Bar(
            y=a[a.Item.str.match("Sawnwood")].Area.values,
            x=a[a.Item.str.match("Sawnwood")].Value.values,
            name='Sawnwood, ' + str(year),
            orientation='h',
            visible=False,
        ),    
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Bar(
            y=a[a.Item.isin(["Pulpwood:round and split:all species (production)"])].Area.values,
            x=a[a.Item.isin(["Pulpwood:round and split:all species (production)"])].Value.values,
            name='Pulpwood, ' + str(year),
            orientation='h',
            visible=False,
        ),    
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Bar(
            y=a[a.Item.str.match('Paper and paperboard')].Area.values,
            x=a[a.Item.str.match('Paper and paperboard')].Value.values * 1.2,
            name='Paper and paperboard, ' + str(year),
            orientation='h',
            visible=False,
        ),    
        row=1,
        col=1,
    )

    b = land_use[land_use.Year == year].groupby(["Area", "Item"]).Value.sum()
    b = b.reset_index()
    b = b[b.Area.str.match("Oceania")].append(b[b.Area.str.match("South America")])\
                                      .append(b[b.Area.str.match("Africa")])\
                                      .append(b[b.Area.str.match("Northern America")])\
                                      .append(b[b.Area.str.match("Europe")])\
                                      .append(b[b.Area.str.match("Asia")])
    fig.add_trace(
        go.Bar(
            y=b.Area.values,
            x=b.Value.values,
            name='Forested area, ' + str(year),
            orientation='h',
            visible=False,
        ),
        row=1,
        col=2,
    )
 

In [28]:
# plot the graph
fig.data[0].visible = True
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True
fig.data[4].visible = True
fig.data[5].visible = True


steps = []
for i in range(int(len(fig.data)/6)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
        label='{}'.format(i + year_min),
    )
    step["args"][1][i*6] = True  # Toggle i'th trace to "visible"
    step["args"][1][i*6+1] = True  # Toggle i'th trace to "visible"
    step["args"][1][i*6+2] = True  # Toggle i'th trace to "visible"
    step["args"][1][i*6+3] = True  # Toggle i'th trace to "visible"
    step["args"][1][i*6+4] = True  # Toggle i'th trace to "visible"
    step["args"][1][i*6+5] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=1991,
    currentvalue={"prefix": "Year: "},
    steps=steps
)]
fig.update_layout(
    sliders=sliders,
    barmode='stack',
    autosize=False,
    width=1000,
    height=600,
)

fig.update_xaxes(range=[0, 2000000000], showgrid=True, row=1, col=1)
fig.update_xaxes(range=[0, 1100000], showgrid=True, row=1, col=2)

fig.show()

## Cut down all over the world
The leadning continent in production primary forest products have been in Asia followed by Northern America nad Europe, but the impact of wood production on defrestation is clearest in South America and Africa. In these two continents has the production increased by almost 30 percent and the area of forested areas decreased by 7, respectively 6 percent. 

In [29]:
# export the plot to html
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
import plotly.graph_objs as go
from IPython.core.display import display, HTML

plot(fig, filename = '1_continent_prod.html')


'1_continent_prod.html'